In [1]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [1]:

# import tweepy
import tweepy as tw

# your Twitter API key and API secret
my_api_key = "*******************************"
my_api_secret = "**************************************************"

# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
api = tw.API(auth, wait_on_rate_limit=True)

search_query = "Budget2024 -filter:retweets"

# get tweets from the API
tweets = tw.Cursor(api.search_tweets,
              q=search_query,
              lang="en").items(1000)
# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
print("Total Tweets fetched:", len(tweets_copy))

import pandas as pd

# intialize the dataframe
tweets_df = pd.DataFrame()
# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)
# show the dataframe
tweets_df.head(len(tweets_copy))
print("Total Tweets stored:", len(tweets_copy))
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./river-tiger-373313-2bf3c879d943.json', scope)
gc = gspread.authorize(credentials)
from df2gspread import df2gspread as d2g
wks_name = 'twitter_data1'
spreadsheet_key = '1edy0It2Ohg6YQzOgBYKKjq4F77bI86PWQ7SpbZuToHY'
d2g.upload(tweets_df.head(len(tweets_copy)), spreadsheet_key, wks_name, credentials=credentials, row_names=True)

Forbidden: 403 Forbidden
200 - Forbidden.